<a href="https://colab.research.google.com/github/XingxinHE/Chief_Curator_Griffin/blob/master/06_Griffin_detect_the_faces_from_artworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade azure-cognitiveservices-vision-face

     |████████████████████████████████| 71kB 1.9MB/s 
     |████████████████████████████████| 92kB 4.6MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
import os, shutil
os.chdir("/content/gdrive/My Drive/Kaggle/Rijksmuseum")

Mounted at /content/gdrive


In [3]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, SnapshotObjectType, OperationStatusType
import cv2

## Authenticate the client


In [9]:
# Set the FACE_SUBSCRIPTION_KEY environment variable with your key as the value.
# This key will serve all examples in this document.
KEY = '7315ddcaddd14366a2ad88b646074aa0'

# Set the FACE_ENDPOINT environment variable with the endpoint from your Face service in Azure.
# This endpoint will be used in all examples in this quickstart.
ENDPOINT = 'https://xxh-face.cognitiveservices.azure.com'

In [10]:
# Create an authenticated FaceClient.
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

## Select Images to Crop(Data Cleaning, don't run second time)

In [80]:
ls

 0000002_SK-A-4877-0.jpg
'01#Loading Info From XML'
'02#Discover Interesting Data'
'03#Griffin Try to distinguish the material of art'
'03#Griffin Try to distinguish the technique of art'
'03#Griffin Try to distinguish the type of art_batch'
'04#Griffin_distinguish the period of art'
 05_Spectrum_of_Art.ipynb
 100k_images_features.p
 date_df.csv
 date_type.csv
 faces/
'[FAIL & ANALYSIS] 03#Griffin Try to distinguish the type of art'
 full_pca_features.p
 gan_img.csv
 griffin_guess_material_oil_binary.h5
 griffin_guess_mtl_training.csv
 guess_mtl_oil_binary.csv
 jpg/
 material_df_binary_oil_recognition.csv
 material_df.csv
 material_df_multlclass_singlelabel_average_amounts.csv
 material_single_label_df.csv
 oil_painting_df.csv
 peroid_df_for_training.csv
 project_curator.csv
 Rembrandt
 tech_df.csv
 tech_df_single_label.csv
 test_img/
 type_df.csv
 xml/


In [ ]:
import pandas as pd

In [147]:
date_type_df = pd.read_csv('date_type.csv', index_col=0)
date_type_df.head()

,date,tag,date_str_len,type,prent,tekening,boekillustratie,ornament,schilderij,historie,foto,start_year,end_year,periods_year,historical_period
0,1644-1648,0000001_SK-A-4878.jpg,9,schilderij,False,False,False,False,True,False,False,1644,1648,4,Early Modern
1,1857-1857,0000002_SK-A-4877.jpg,9,schilderij,False,False,False,False,True,False,False,1857,1857,0,Modern
2,1675-1685,0000003_SK-A-4881.jpg,9,schilderij,False,False,False,False,True,False,False,1675,1685,10,Early Modern
3,1555-1556,0000004_RP-P-1992-35.jpg,9,"['prent', 'historieprent']",True,False,False,False,False,True,False,1555,1556,1,Renaissance
4,1555-1556,0000005_RP-P-1992-36.jpg,9,"['prent', 'historieprent']",True,False,False,False,False,True,False,1555,1556,1,Renaissance


In [157]:
temp1 = list(date_type_df[date_type_df['prent']==True].index)
temp2 = list(date_type_df[date_type_df['tekening']==True].index)
temp3 = list(date_type_df[date_type_df['boekillustratie']==True].index)
temp4 = list(date_type_df[date_type_df['schilderij']==True].index)
filter_index = set(temp1 + temp2 + temp3 + temp4)
len(filter_index)

94932

In [166]:
date_type_df = date_type_df[date_type_df.index.isin(filter_index)]
date_type_df.reset_index(drop=True, inplace=True)
date_type_df

,date,tag,date_str_len,type,prent,tekening,boekillustratie,ornament,schilderij,historie,foto,start_year,end_year,periods_year,historical_period
0,1644-1648,0000001_SK-A-4878.jpg,9,schilderij,False,False,False,False,True,False,False,1644,1648,4,Early Modern
1,1857-1857,0000002_SK-A-4877.jpg,9,schilderij,False,False,False,False,True,False,False,1857,1857,0,Modern
2,1675-1685,0000003_SK-A-4881.jpg,9,schilderij,False,False,False,False,True,False,False,1675,1685,10,Early Modern
3,1555-1556,0000004_RP-P-1992-35.jpg,9,"['prent', 'historieprent']",True,False,False,False,False,True,False,1555,1556,1,Renaissance
4,1555-1556,0000005_RP-P-1992-36.jpg,9,"['prent', 'historieprent']",True,False,False,False,False,True,False,1555,1556,1,Renaissance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94927,1830-1830,0112378_RP-P-OB-86.307.jpg,9,prent,True,False,False,False,False,False,False,1830,1830,0,Modern
94928,1608-1615,0112379_RP-P-2012-4.jpg,9,prent,True,False,False,False,False,False,False,1608,1615,7,Early Modern
94929,1800-1900,0112380_RP-P-2011-173-1.jpg,9,prent,True,False,False,False,False,False,False,1800,1900,100,Early Modern
94930,1652-1653,0112381_RP-T-1886-A-704-26.jpg,9,tekening,False,True,False,False,False,False,False,1652,1653,1,Early Modern


In [167]:
temp5 = list(date_type_df[date_type_df['historical_period']=='Renaissance'].index)
temp6 = list(date_type_df[date_type_df['historical_period']=='Early Modern'].index)

time_filter_index = set(temp5 + temp6)
len(time_filter_index)

88193

In [168]:
date_type_df = date_type_df[date_type_df.index.isin(time_filter_index)]
date_type_df.reset_index(drop=True, inplace=True)
date_type_df

,date,tag,date_str_len,type,prent,tekening,boekillustratie,ornament,schilderij,historie,foto,start_year,end_year,periods_year,historical_period
0,1644-1648,0000001_SK-A-4878.jpg,9,schilderij,False,False,False,False,True,False,False,1644,1648,4,Early Modern
1,1675-1685,0000003_SK-A-4881.jpg,9,schilderij,False,False,False,False,True,False,False,1675,1685,10,Early Modern
2,1555-1556,0000004_RP-P-1992-35.jpg,9,"['prent', 'historieprent']",True,False,False,False,False,True,False,1555,1556,1,Renaissance
3,1555-1556,0000005_RP-P-1992-36.jpg,9,"['prent', 'historieprent']",True,False,False,False,False,True,False,1555,1556,1,Renaissance
4,1553-1553,0000006_RP-P-1992-68.jpg,9,prent,True,False,False,False,False,False,False,1553,1553,0,Renaissance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88188,1680-1695,0112371_RP-P-2011-65-30.jpg,9,prent,True,False,False,False,False,False,False,1680,1695,15,Early Modern
88189,1680-1695,0112372_RP-P-2011-65-31.jpg,9,prent,True,False,False,False,False,False,False,1680,1695,15,Early Modern
88190,1608-1615,0112379_RP-P-2012-4.jpg,9,prent,True,False,False,False,False,False,False,1608,1615,7,Early Modern
88191,1800-1900,0112380_RP-P-2011-173-1.jpg,9,prent,True,False,False,False,False,False,False,1800,1900,100,Early Modern


In [169]:
date_type_df.to_csv('renaissance_early_modern_drawings.csv')

## Import cleaned CSV

In [ ]:
date_type_df = pd.read_csv('renaissance_early_modern_drawings.csv', index_col=0)

In [170]:
date_type_df = date_type_df.loc[:, ['tag','type','historical_period']]
date_type_df

,tag,type,historical_period
0,0000001_SK-A-4878.jpg,schilderij,Early Modern
1,0000003_SK-A-4881.jpg,schilderij,Early Modern
2,0000004_RP-P-1992-35.jpg,"['prent', 'historieprent']",Renaissance
3,0000005_RP-P-1992-36.jpg,"['prent', 'historieprent']",Renaissance
4,0000006_RP-P-1992-68.jpg,prent,Renaissance
...,...,...,...
88188,0112371_RP-P-2011-65-30.jpg,prent,Early Modern
88189,0112372_RP-P-2011-65-31.jpg,prent,Early Modern
88190,0112379_RP-P-2012-4.jpg,prent,Early Modern
88191,0112380_RP-P-2011-173-1.jpg,prent,Early Modern


In [171]:
date_type_df['file_name'] = '/content/gdrive/My Drive/Kaggle/Rijksmuseum/jpg/' +date_type_df['tag']
date_type_df.head()

,tag,type,historical_period,file_name
0,0000001_SK-A-4878.jpg,schilderij,Early Modern,/content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
1,0000003_SK-A-4881.jpg,schilderij,Early Modern,/content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
2,0000004_RP-P-1992-35.jpg,"['prent', 'historieprent']",Renaissance,/content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
3,0000005_RP-P-1992-36.jpg,"['prent', 'historieprent']",Renaissance,/content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
4,0000006_RP-P-1992-68.jpg,prent,Renaissance,/content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...


In [172]:
file_list = date_type_df['file_name']
file_list[13]

0    /content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
1    /content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
2    /content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
3    /content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
4    /content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
5    /content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
6    /content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
7    /content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
8    /content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
9    /content/gdrive/My Drive/Kaggle/Rijksmuseum/jp...
Name: file_name, dtype: object

## Crop Face Function

In [174]:
#重要
scale_factor = 1.5

In [217]:
def crop_face(file_path):

  read_image = cv2.imread(file_path)
  w = open(file_path, 'r+b')
  detected_faces = face_client.face.detect_with_stream(w)
  if detected_faces:
    for i, face in enumerate(detected_faces):
      print(i, face)
      rect = face.face_rectangle
      left = rect.left
      top = rect.top
      right = left + rect.width
      bottom = top + rect.height
      width = rect.width
      new_width = round(rect.width*scale_factor)

      if width%2 != 0:
        width+=1

      if new_width%2 != 0:
        new_width+=1
      
      delta = int((new_width - width)/2)

      #怕框超出界
      bool1 = (top-delta)<0
      bool2 = (left-delta)<0
      bool3 = (bottom+delta)>read_image.shape[0]
      bool4 = (right+delta)>read_image.shape[1]

      if(bool1 or bool2 or bool3 or bool4):
        min_value = min(top-delta, left-delta, read_image.shape[0]-(bottom+delta), read_image.shape[1]-(right+delta))
        delta = delta+min_value

      face_pix = read_image[top-delta:bottom+delta, left-delta:right+delta, :]
      face_pix = cv2.resize(face_pix, (128, 128))
      save_filename = '%s-%d.jpg' % (os.path.basename(file_path).split('.')[0], i)
      cv2.imwrite("faces/Renaissance/" + save_filename, face_pix)
  else:
    print('No face detected from image {}'.format(os.path.basename(file_path)))

## Operate Crop Function

In [219]:
import time
starttime = time.time()
i=2817
while True:
  print("tick: processing \t"+str(i)+"\t piece of file")
  crop_face(file_list[i])
  i+=1
  #time.sleep(7 - time.time() % 7)

tick: processing 	2280	 piece of file
No face detected from image 0004252_SK-A-4035.jpg
tick: processing 	2281	 piece of file
No face detected from image 0004253_SK-A-1965.jpg
tick: processing 	2282	 piece of file
0 {'additional_properties': {}, 'face_id': '50cb2cdd-9b01-4ae7-bf96-d15a0c05fcc9', 'recognition_model': None, 'face_rectangle': <azure.cognitiveservices.vision.face.models._models_py3.FaceRectangle object at 0x7fe026cac8d0>, 'face_landmarks': None, 'face_attributes': None}
tick: processing 	2283	 piece of file
0 {'additional_properties': {}, 'face_id': 'ab271841-3e57-456c-b7a3-11cdfe8fadf5', 'recognition_model': None, 'face_rectangle': <azure.cognitiveservices.vision.face.models._models_py3.FaceRectangle object at 0x7fe026cacd30>, 'face_landmarks': None, 'face_attributes': None}
tick: processing 	2284	 piece of file
0 {'additional_properties': {}, 'face_id': '3bccf2b4-ab3e-443c-ae78-c4d0e101ac02', 'recognition_model': None, 'face_rectangle': <azure.cognitiveservices.vision.fa

KeyboardInterrupt: ignored

In [184]:
file_list[547] 883 1268 2279	 

'/content/gdrive/My Drive/Kaggle/Rijksmuseum/jpg/0002188_SK-C-404.jpg'